In [38]:
import re
from collections import defaultdict

In [85]:
class BytePairEncoder:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.vocabulary = None
        self.id2voc = None
        self.merges = []

    # Corpus, string olmalı
    def train(self, corpus):

        if isinstance(corpus, list) and all(isinstance(item, str) for item in corpus):
            buff = ""
            for item in corpus:
                buff += item + " . "
            corpus = buff
            print(corpus)
            del buff
        
        chars = sorted(list(set(corpus)))
        vocabulary = {}
        id2voc = {}
        
        for idx, char in enumerate(chars):
            vocabulary[char] = idx

        for char, idx in vocabulary.items():
            id2voc[idx] = char
        
        token_ids = [vocabulary[char] for char in corpus]
        current_id_len = len(chars)

        while current_id_len < self.vocab_size:
            pairs = self.get_pairs(token_ids)
            
            if not pairs:
                break  # birleştirilecek çift yoksa loop u durdur
                
            best_pair = max(pairs, key=lambda x: pairs[x]) # En yüksek frekanslı çifti al
            a, b = best_pair
            new_token = id2voc[a] + id2voc[b]
            
            vocabulary[new_token] = current_id_len
            id2voc[current_id_len] = new_token
            self.merges.append((best_pair, current_id_len))
            
            token_ids = self.merge_pair(token_ids, best_pair, current_id_len)
            current_id_len += 1

        self.vocabulary = vocabulary
        self.id2voc = id2voc
        return self.vocabulary, self.id2voc

    def get_pairs(self, token_ids):
        pairs = defaultdict(int) # başlangıç valueları 0 olan bir sözlük oluşturur
        for i in range(len(token_ids)-1):
            pair = (token_ids[i], token_ids[i+1])
            pairs[pair] += 1
        return pairs

    def merge_pair(self, token_ids, pair, new_id):
        new_token_ids = []
        i = 0
        while i < len(token_ids):
            if i < len(token_ids)-1 and (token_ids[i], token_ids[i+1]) == pair:
                new_token_ids.append(new_id)
                i += 2
            else:
                new_token_ids.append(token_ids[i])
                i += 1
        return new_token_ids

    def encode(self, text):
        tokens = list(text)
        token_ids = []
        for char in tokens:
            token_ids.append(self.vocabulary.get(char, 0)) # unk = 0 = ' '
        
        for (pair, new_id) in self.merges:
            token_ids = self.merge_pair(token_ids, pair, new_id)
        
        return token_ids

    def decode(self, token_ids):
        return ''.join([self.id2voc[idx] for idx in token_ids])

In [87]:
corpus = [
    "Yine geldi def bumba gibi",
    "Merhaba dünya"
]
bpe = BytePairEncoder(vocab_size=25)
bpe.train(corpus)

print("Öğrenilen Birleştirmeler:", bpe.merges)
print("Öğrenilen Sözlük:", bpe.vocabulary)

text_to_encode = "merhaba dünya yeni bir deneme"
print("Kodlanmış Metin:", bpe.encode(text_to_encode))
print("Çözülmüş Metin:", bpe.decode(bpe.encode(text_to_encode)))

Yine geldi def bumba gibi . Merhaba dünya . 
Öğrenilen Birleştirmeler: [((4, 0), 19), ((11, 0), 20), ((5, 19), 21), ((1, 0), 22), ((3, 11), 23), ((23, 14), 24)]
Öğrenilen Sözlük: {' ': 0, '.': 1, 'M': 2, 'Y': 3, 'a': 4, 'b': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'l': 12, 'm': 13, 'n': 14, 'r': 15, 'u': 16, 'y': 17, 'ü': 18, 'a ': 19, 'i ': 20, 'ba ': 21, '. ': 22, 'Yi': 23, 'Yin': 24}
Kodlanmış Metin: [13, 7, 15, 10, 4, 21, 6, 18, 14, 17, 19, 17, 7, 14, 20, 5, 11, 15, 0, 6, 7, 14, 7, 13, 7]
Çözülmüş Metin: merhaba dünya yeni bir deneme
